In [1]:
import pandas as pd
from json import load
import numpy as np
from ucimlrepo import fetch_ucirepo
import psycopg2
import os


In [17]:
secret_path = r'D:\CS\Data Science\Case Study\src\secrets\postgres.json'
with open(secret_path, 'r') as file:
    settings = load(file)

table = settings['table']
host = settings['host']
dbname = settings['dbname']
user = settings['user']
password = settings['password']

conn = psycopg2.connect(f'host={host} dbname={dbname} user={user} password={password}')


In [18]:
# Taken from:
# https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/

def copy_from_file(conn, df, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file  
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    tmp_df = "./tmp_dataframe.csv"
    df.to_csv(tmp_df, index_label='id', header=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    # os.remove(tmp_df)

In [4]:
adult_income = fetch_ucirepo(id=2)

X = adult_income.data.features
y = adult_income.data.targets

In [24]:
income_series = pd.Series(y.income, name='income')
combined_df = pd.concat([X, income_series], axis=1)
# Replace null values from dataset to something SQL readable
combined_df.replace('?', float('nan'), inplace=True)


In [25]:
# copy_from_file(conn=conn, df=combined_df, table=table)

copy_from_file() done
